In [1]:
# Imports 
import os
import sys
from io import StringIO
sys.path.append("..")
os.environ["pyna_language"] = 'julia'
import pdb
import numpy as np
import openmdao.api as om
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
%matplotlib inline
from pyNA.pyna import pyna

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Inputs 
z_cb_lst = np.hstack((np.linspace(25, 250, 10), np.linspace(300, 750, 10)))

TS_cb_lst = dict()
TS_cb_lst['stca']= dict()
TS_cb_lst['stca']['V2'] = np.linspace(0.75, 1.0, 11)
TS_cb_lst['stca']['Vmax'] = np.linspace(0.4, 1.0, 25)
TS_cb_lst['a10'] = dict()
TS_cb_lst['a10']['V2'] = np.linspace(0.775, 1.0, 10)
TS_cb_lst['a10']['Vmax'] = np.linspace(0.675, 1.0, 14)

Vmax_lst = dict()
Vmax_lst['stca'] = dict()
Vmax_lst['stca']['V2'] = 197*0.51444444
Vmax_lst['stca']['Vmax'] = 250*0.51444444
Vmax_lst['a10'] = dict()
Vmax_lst['a10']['V2'] = 234.4*0.51444444
Vmax_lst['a10']['Vmax'] = 250*0.51444444

py = pyna()
limits_stca = py.get_icao_annex_16_noise_limits(mtow=np.array([55.]), chapter='14', n_eng=3)
limits_a10 = py.get_icao_annex_16_noise_limits(mtow=np.array([166.5]), chapter='14', n_eng=3)

epnl_constraint = dict()
epnl_constraint['stca'] = dict() 
epnl_constraint['stca']['V2']   = limits_stca['lateral'][0]-0.5
epnl_constraint['stca']['Vmax'] = limits_stca['lateral'][0]-0.5
epnl_constraint['a10'] = dict()
epnl_constraint['a10']['V2']   = limits_a10['lateral'][0]-0.5
epnl_constraint['a10']['Vmax'] = limits_a10['lateral'][0]-0.5

# PHLD trajectories

In [ ]:
# Load verification trajectory 
nasa_std = pd.read_csv('../cases/nasa_stca_standard/trajectory/Trajectory_to.csv')

if True:
    
    for ac_name in ['stca']:
        for ii, case in enumerate(['V2', 'Vmax']):

            py = pyna()
            py.initialize()
            
            if True:

                py.ac_name = ac_name
                py.case_name = ac_name
                py.engine_deck_name = 'engine_deck_'+ac_name+'.csv'
                py.TS_to = 1.
                py.TS_vnrs = TS_cb_lst[ac_name][case][0]
                py.TS_cb = TS_cb_lst[ac_name][case][0]
                py.save_results = True
                py.levels_int_metric = 'epnl'
                py.output_directory_name = 'PHLD/'+case
                py.output_file_name = ac_name+'_phld_'+case+'.sql'

                py.noise_constraint_lateral = epnl_constraint[ac_name][case]
                py.ptcb = True
                py.pkrot = True
                py.phld = True
                py.v_max = Vmax_lst[ac_name][case]
                
                py.all_sources = False
                py.fan_inlet_source = True
                py.fan_discharge_source = True
                py.core_source = True
                py.jet_mixing_source = True
                py.jet_shock_source = False
                py.airframe_source = False

                # Simplifications
                py.n_frequency_subbands = 1
                py.max_iter = 400

                if ac_name == 'stca' and case == 'V2':
                    x_observer_array = np.linspace(1000, 6250, 16)
                elif ac_name == 'stca' and case == 'Vmax':
                    x_observer_array = np.linspace(1000, 6250, 16)
                elif ac_name == 'a10' and case == 'V2':
                    x_observer_array = np.linspace(2300, 6500, 13)
                elif ac_name == 'a10' and case == 'Vmax':
                    x_observer_array = np.linspace(2300, 6500, 13)

            if True:                  
                py.optimize_trajectory_noise(x_lateral_observer=x_observer_array, initialization_path_name=None)

            if True:
                results = py.load_results(file_name=ac_name+'_phld_'+case+'.sql')

                path_save_name = 'trajectory_' + ac_name + '_phld_' + case + '.csv'
                engine_save_name = 'engine_' + ac_name + '_phld_' + case + '.csv' 
                py.save_time_series(results, py.airframe, path_save_name, engine_save_name)

            if True:
                n_sl = 131  

                py.engine_timeseries_name = 'engine_'+ac_name+'_phld_'+case+'.csv'
                py.trajectory_file_name = 'trajectory_'+ac_name+'_phld_'+case+'.csv'
                py.output_file_name = ac_name+'_phld_'+case+'_noisetimeseries.sql'
                py.observer_lst = ['contour',]
                py.levels_int_metric = 'epnl'

                py.all_sources = False
                py.fan_inlet_source = True
                py.fan_discharge_source = True
                py.core_source = True
                py.jet_mixing_source = True
                py.jet_shock_source = False
                py.airframe_source = True

                x_observer_lst = np.zeros((n_sl+1, 3))
                x_observer_lst[:-1, 0] = np.linspace(0, 6500, n_sl)
                x_observer_lst[:-1, 1] = 450*np.ones(n_sl)
                x_observer_lst[:, 2] = 4*0.3048*np.ones(n_sl+1)
                x_observer_lst[-1,0] = 6500.

                py.x_observer_array = x_observer_lst
                py.compute_noise_timeseries()

                epnl_sideline = py.noise_timeseries.get_val('noise.epnl')[:-1]
                epnl_lateral = np.max(py.noise_timeseries.get_val('noise.epnl')[:-1])
                epnl_flyover = py.noise_timeseries.get_val('noise.epnl')[-1]

            if True:
                np.save('../cases/'+ac_name+'/output/PHLD/'+case+'/epnl_lateral.npy', epnl_lateral)
                np.save('../cases/'+ac_name+'/output/PHLD/'+case+'/epnl_flyover.npy', epnl_flyover)
                np.save('../cases/'+ac_name+'/output/PHLD/'+case+'/epnl_sideline.npy', epnl_sideline)

/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/utils/options_dictionary.py:332: OMDeprecationWarning:'gradient method' is not a valid python name and will become an invalid option name in a future release. You can prevent this warning (and future exceptions) by declaring this option using a valid python name.


--- Linkage Report [phases] ---
    --- groundroll - rotation ---
        time           [final] ==  time           [initial]
        states:x       [final] ==  states:x       [initial]
        states:v       [final] ==  states:v       [initial]
        states:alpha   [final] ==  states:alpha   [initial]
    --- rotation - liftoff ---
        time               [final] ==  time             [initial]
        states:x           [final] ==  states:x         [initial]
        parameters:z       [final*] ==  states:z         [initial]
        states:v           [final] ==  states:v         [initial]
        states:alpha       [final] ==  controls:alpha   [initial]
        parameters:gamma   [final*] ==  states:gamma     [initial]
    --- liftoff - vnrs ---
        time             [final] ==  time             [initial]
        states:x         [final] ==  states:x         [initial]
        states:v         [final] ==  states:v         [initial]
        controls:alpha   [final] ==  controls:

/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/recorders/sqlite_recorder.py:224: UserWarning:The existing case recorder file, dymos_solution.db, is being overwritten.


Model viewer data has already has already been recorded for Driver.
Model viewer data has already has already been recorded for Driver.
Full total jacobian was computed 3 times, taking 6.446835 seconds.
Total jacobian shape: (255, 179) 


Jacobian shape: (255, 179)  ( 3.77% nonzero)
FWD solves: 11   REV solves: 0
Total colors vs. total size: 11 vs 179  (93.9% improvement)

Sparsity computed using tolerance: 1e-12
Time to compute sparsity: 6.446835 sec.
Time to compute coloring: 0.241306 sec.


Optimization Problem -- Optimization using pyOpt_sparse
    Objective Function: _objfunc

    Solution: 
--------------------------------------------------------------------------------
    Total Time:                   18.8738
       User Objective Time :       2.3054
       User Sensitivity Time :    14.8554
       Interface Time :            1.5601
       Opt Solver Time:            0.1529
    Calls to Objective Function :      35
    Calls to Sens Function :           35


   Objectives
     

--- Linkage Report [phases] ---
    --- groundroll - rotation ---
        time                     [final] ==  time                     [initial]
        states:x                 [final] ==  states:x                 [initial]
        states:v                 [final] ==  states:v                 [initial]
        states:alpha             [final] ==  states:alpha             [initial]
        parameters:theta_flaps   [final*] ==  parameters:theta_flaps   [initial*]
    --- rotation - liftoff ---
        time                     [final] ==  time                     [initial]
        states:x                 [final] ==  states:x                 [initial]
        parameters:z             [final*] ==  states:z                 [initial]
        states:v                 [final] ==  states:v                 [initial]
        states:alpha             [final] ==  controls:alpha           [initial]
        parameters:gamma         [final*] ==  states:gamma             [initial]
        parameters:

/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/recorders/sqlite_recorder.py:224: UserWarning:The existing case recorder file, /Users/laurensvoet/Documents/Research/pyNA/pyNA/cases/stca/output/PHLD/Vmax/stca_phld_Vmax.sql, is being overwritten.


Model viewer data has already has already been recorded for Driver.
Model viewer data has already has already been recorded for Driver.
Full total jacobian was computed 3 times, taking 712.697480 seconds.
Total jacobian shape: (266, 197) 


Jacobian shape: (266, 197)  ( 4.44% nonzero)
FWD solves: 14   REV solves: 2
Total colors vs. total size: 16 vs 197  (91.9% improvement)

Sparsity computed using tolerance: 1e-12
Time to compute sparsity: 712.697480 sec.
Time to compute coloring: 0.537173 sec.
